# 🌟 Lightkurve Tutorial for Kepler, K2, and TESS

This notebook demonstrates how to use the [Lightkurve](https://docs.lightkurve.org) Python library to search, download, and analyze light curves from NASA's **Kepler**, **K2**, and **TESS** missions.

## 📚 What is Lightkurve?

Lightkurve is a high-level Python package for working with astronomical time series data, especially from the **Kepler**, **K2**, and **TESS** space telescopes. It provides easy tools for:

- Downloading light curve data from the MAST archive
- Cleaning and normalizing light curves
- Visualizing and folding periodic signals
- Performing period analysis (e.g., BLS, Lomb-Scargle)
- Working with target pixel files (TPFs)

> 📦 Install using: `pip install lightkurve`

---


In [ ]:
import lightkurve as lk
import numpy as np
import matplotlib.pyplot as plt


## 🔧 Lightkurve Command Reference

### `lk.search_lightcurve(target, mission=...)`
Searches the MAST archive for light curve data.
- `target`: Object name or TIC/EPIC ID.
- `mission`: `"Kepler"`, `"K2"`, or `"TESS"`.
- `author`: Optional; e.g., `"SPOC"` or `"QLP"` for TESS.

### `download()` and `download_all()`
- `.download()`: Downloads the first matching light curve.
- `.download_all()`: Downloads all light curves found.

### `stitch()`
Combines multiple segments of a light curve into one continuous light curve.

### `plot()`
Visualizes the light curve, periodogram, or target pixel file.
- Customize using `title`, `label`, `color`, etc.

### `remove_nans()`, `remove_outliers(sigma=...)`, `normalize()`
Cleans the light curve:
- Removes NaNs
- Removes outliers (default `sigma=5`)
- Normalizes so median flux = 1

### `flatten(window_length, polyorder)`
Removes long-term trends using Savitzky-Golay filter.

### `to_periodogram(method=...)`
Computes power spectrum of periodic signals:
- `"lombscargle"` for continuous variability
- `"bls"` for transits

### `fold(period)`
Folds the light curve at a given period to show repeated patterns.

### `search_targetpixelfile(target, mission=...)`
Gets pixel-level data (TPFs).

### `create_threshold_mask(threshold=...)`
Generates a mask to extract relevant pixels above flux threshold.

### `to_lightcurve(aperture_mask=...)`
Extracts light curve from pixel data using the given mask.

### `to_fits()` and `to_csv()`
Exports light curve to `.fits` or `.csv` format.

### `interact()`
Jupyter widget to interactively explore light curves or TPFs.

---

These commands form the backbone of data analysis using Lightkurve. You can chain many of them together for efficient pipelines.


In [ ]:
# 🔍 1. Search Light Curve Examples
kepler_search = lk.search_lightcurve("Kepler-10", mission="Kepler")
k2_search = lk.search_lightcurve("EPIC 201367065", mission="K2")
tess_search = lk.search_lightcurve("TIC 25155310", mission="TESS", author="SPOC")

# Display search results
print("Kepler:", kepler_search)
print("K2:", k2_search)
print("TESS:", tess_search)


In [ ]:
# 📥 2. Download and Stitch Light Curves
kepler_lc = kepler_search.download_all().stitch()
k2_lc = k2_search.download_all().stitch()
tess_lc = tess_search.download_all().stitch()

# 📈 3. Plot Raw Light Curves
kepler_lc.plot(title="Kepler-10 Raw Light Curve")
k2_lc.plot(title="EPIC 201367065 Raw Light Curve")
tess_lc.plot(title="TIC 25155310 Raw Light Curve")


In [ ]:
# 🧹 4. Clean and Normalize
kepler_clean = kepler_lc.remove_nans().remove_outliers(sigma=5).normalize()
k2_clean = k2_lc.remove_nans().remove_outliers(sigma=5).normalize()
tess_clean = tess_lc.remove_nans().remove_outliers(sigma=5).normalize()

kepler_clean.plot(title="Kepler-10 Cleaned Light Curve")


In [ ]:
# 🧵 5. Flatten Light Curve
kepler_flat = kepler_clean.flatten(window_length=401, polyorder=2)
kepler_flat.plot(title="Kepler-10 Flattened Light Curve")


In [ ]:
# 📊 6. Periodogram Analysis
kepler_pg = kepler_flat.to_periodogram()
kepler_pg.plot(title="Kepler-10 Lomb-Scargle Periodogram")
best_period = kepler_pg.period_at_max_power
print(f"Best Period: {best_period}")


In [ ]:
# 🔁 7. Fold Light Curve
kepler_folded = kepler_flat.fold(period=best_period)
kepler_folded.plot(title="Kepler-10 Folded Light Curve")


In [ ]:
# 🔍 8. Target Pixel File
kepler_tpf = lk.search_targetpixelfile("Kepler-10", mission="Kepler").download()
kepler_tpf.plot(title="Kepler-10 Target Pixel File")


In [ ]:
# 🎯 9. Custom Aperture Mask
custom_mask = kepler_tpf.create_threshold_mask(threshold=5)
kepler_tpf.plot(aperture_mask=custom_mask, title="Custom Aperture Mask")


In [ ]:
# 💡 10. Extract Custom Light Curve
custom_lc = kepler_tpf.to_lightcurve(aperture_mask=custom_mask)
custom_lc.plot(title="Custom Light Curve from TPF")


In [ ]:
# 🧪 11. Transit Search with BLS
bls_pg = kepler_flat.to_periodogram(method='bls', period=np.linspace(0.5, 10, 10000))
bls_pg.plot(title="Kepler-10 BLS Periodogram")


In [ ]:
# 📂 12. Save to File
kepler_flat.to_fits("kepler10_flat_lightcurve.fits")
kepler_flat.to_csv("kepler10_flat_lightcurve.csv")


In [ ]:
# 🧠 13. Interactives (Run in Jupyter Notebook)
# kepler_lc.interact()
# kepler_tpf.interact()

print("Tutorial complete!")
